In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [4]:
# def cov_matrix (n, sigma2, rho):
#       m = matrix(nrow = n, ncol = n)
#       m[upper.tri(m)] = rho
#       m[lower.tri(m)] = rho
#       diag(m) = 1
#       (sigma2^.5 * diag(n))  %*% m %*% (sigma2^.5 * diag(n))


In [6]:
## Master variables
RUN_DATE = datetime.strptime("2008-11-03", '%Y-%m-%d')

election_day = datetime.strptime("2008-11-03", '%Y-%m-%d')
start_date = datetime.strptime("2008-03-01", '%Y-%m-%d') # Keeping all polls after March 1, 2008

In [11]:
df_2008 = pd.read_csv('../data/raw/all_polls_2008_2.csv', parse_dates=['start.date', 'end.date'])

In [12]:
df_2008.head()

,state,pollster,number.of.observations,start.date,end.date,obama,mccain,other,undecided,mode,population
0,AK,SurveyUSA,600.0,2008-03-03,2008-03-06,43.0,48.0,6.3,2.7,Live Phone,Registered Voters
1,AK,Rasmussen,500.0,2008-05-14,2008-05-17,41.0,50.0,6.3,2.7,Live Phone,Registered Voters
2,AK,Rasmussen,500.0,2008-06-15,2008-06-18,41.0,45.0,9.8,4.2,Live Phone,Registered Voters
3,AK,Research 2000,600.0,2008-07-13,2008-07-16,41.0,51.0,5.6,2.4,Live Phone,Registered Voters
4,AK,Rasmussen,500.0,2008-07-18,2008-07-21,40.0,45.0,10.5,4.5,Live Phone,Registered Voters


In [13]:
# select relevant columns from HufFPost polls
# all_polls <- all_polls %>%
#   dplyr::select(state, pollster, number.of.observations, mode,population,
#                 start.date, 
#                 end.date,
#                 obama, mccain, undecided, other)%>%
#   mutate(end.date = mdy(end.date),
#          start.date = mdy(start.date))

df_2008 = df_2008[['state', 'pollster', 'number.of.observations', 'mode', 'population', 
                   'start.date', 'end.date', 'obama', 'mccain', 'undecided', 'other']]

In [14]:
# # make sure we've got nothing from the futuree
# all_polls <- all_polls %>%
#   filter(ymd(end.date) <= RUN_DATE)

df_2008 = df_2008[df_2008['end.date'] <= RUN_DATE]

In [23]:
# # basic mutations
# df <- all_polls %>% 
#   tbl_df %>%
#   rename(n = number.of.observations) %>%
#   mutate(begin = ymd(start.date),
#          end   = ymd(end.date),
#          t = end - (1 + as.numeric(end-begin)) %/% 2) %>%
#   filter(t >= start_date & !is.na(t)
#          & n > 1) 

# pollster mutations
# df <- df %>%
#   mutate(pollster = str_extract(pollster, pattern = "[A-z0-9 ]+") %>% sub("\\s+$", "", .),
#          pollster = replace(pollster, pollster == "Fox News", "FOX"), # Fixing inconsistencies in pollster names
#          pollster = replace(pollster, pollster == "WashPost", "Washington Post"),
#          pollster = replace(pollster, pollster == "ABC News", "ABC"),
#          undecided = ifelse(is.na(undecided), 0, undecided),
#          other = ifelse(is.na(other), 0, other))


df_2008['pollster'] = df_2008['pollster'].str.replace("Fox News", 'FOX')
df_2008['pollster'] = df_2008['pollster'].str.replace("WashPost", 'Washington Post')
df_2008['pollster'] = df_2008['pollster'].str.replace("ABC News", 'ABC')
df_2008['undecided'] = df_2008['undecided'].fillna(0)
df_2008['other'] = df_2008['other'].fillna(0)




<ipython-input-23-4f6fc8f11f90>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008['pollster'] = df_2008['pollster'].str.replace("Fox News", 'FOX')
<ipython-input-23-4f6fc8f11f90>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008['pollster'] = df_2008['pollster'].str.replace("WashPost", 'Washington Post')
<ipython-input-23-4f6fc8f11f90>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [35]:
df_2008.head()

,state,pollster,number.of.observations,mode,population,start.date,end.date,obama,mccain,undecided,other,two_party_sum,polltype
0,AK,SurveyUSA,600.0,Live Phone,Registered Voters,2008-03-03,2008-03-06,43.0,48.0,2.7,6.3,91.0,1
1,AK,Rasmussen,500.0,Live Phone,Registered Voters,2008-05-14,2008-05-17,41.0,50.0,2.7,6.3,91.0,1
2,AK,Rasmussen,500.0,Live Phone,Registered Voters,2008-06-15,2008-06-18,41.0,45.0,4.2,9.8,86.0,1
3,AK,Research 2000,600.0,Live Phone,Registered Voters,2008-07-13,2008-07-16,41.0,51.0,2.4,5.6,92.0,1
4,AK,Rasmussen,500.0,Live Phone,Registered Voters,2008-07-18,2008-07-21,40.0,45.0,4.5,10.5,85.0,1


In [34]:
# # vote shares etc
# df <- df %>%
#   mutate(two_party_sum = obama + mccain,
#          polltype = as.integer(as.character(recode(population, 
#                                                    "Likely Voters" = "0", 
#                                                    "Registered Voters" = "1",
#                                                    "Adults" = "2"))), 
#          n_respondents = round(n),
#          # obama
#          n_obama = round(n * obama/100),
#          p_obama = obama/two_party_sum,
#          # mccain
#          n_mccain = round(n * mccain/100),
#          p_mccain = mccain/two_party_sum,
#          # third-party
#          n_other = round(n * other/100),
#          p_other = other/100)

df_2008.loc[:,'two_party_sum'] = df_2008.loc[:, 'obama'] + df_2008.loc[:,'mccain']
df_2008.loc[:,'polltype'] = df_2008.loc[:,'population'].map({"Likely Voters" : 0, "Registered Voters" : 1, "Adults" : 2})

